Streams


1 cons-stream

the return value of (cons-stream a b) is (a,#I promise i would eval b)

1-1 cdr-stream  the cdr of cons-stream

scm> (cons-stream 1 2)
(1 . #[promise (not forced)])

非惰性求值会爆栈
(define (ints first)
    (cons first
        (ints (+ first 1))))

scm> (ints 1)
maximum recursion depth exceeded

(define (ints first)
(cons-stream first
(      ints (+ first 1))))

scm> (ints 1)
(1 . #[promise (not forced)])


(define (search inx x)
 ( if( = inx 1)
(car (cdr-stream x))
 (search (- inx 1) (cdr-stream x))))

2 delay and force

delay不是函数，因为delay的功能是封装表达式而不是输出一个值

scm> (define box5 (delay (print 5)))
box5

scm> box5
#[promise (not forced)]

scm> (force box5)
5

scm> box5
#[promise (forced)]
scm> 


forced 和 not forced :forced表示该封装已经被force过

回看 cdr-stream: 类似于cdr + force

scm> (define a (int 0))
a
scm> a
(0 . #[promise (not forced)])
scm> (cdr-stream a)
(1 . #[promise (not forced)])
scm> (cdr-stream a)
(1 . #[promise (not forced)])
scm> (cdr a)
#[promise (forced)]
scm> (define b (int 0))
b
scm> (force (cdr b))
(1 . #[promise (not forced)])

(define (add-stream s1 s2) 
     (cons-stream ( + ( car s1) (car s2)) (add-stream  (cdr-stream s1) (cdr-stream s2) )))

(define (map-stream  f s)
    (   
        if(null? s)
        nil
        (cons-stream (f (car s))
        (map-stream f (cdr-stream s)))))

 (define (map-stream fn s)
(if (null? s)
nil
(cons-stream (fn (car s))
(map-stream fn (cdr-stream s)))))

scm> (define nums (cons-stream 1 (cons-stream (f 3) (cons-stream (f 5) nil))))
scm> (define (f x) (+ x 1))
scm>  (cdr-stream nums)
4
(define (f x)(+ x 2))
scm>  (cdr-stream nums)
4 ;这里改变了f,但是第二个的stream已经是forced的状态，所以不会再重新计算
scm> (cdr-stream (cdr-stream nums));而第三个的stream是未forced的状态，所以会重新计算
7